In [46]:
import pandas as pd
import requests
import csv

In [98]:
df = pd.read_csv("/Users/natashasebastian/Files/Classes/IV/Project/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2021_20241024.csv")
df.head()

/var/folders/km/qj81chcs4x9bh4f53fts8wvw0000gn/T/ipykernel_80793/2735027698.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/natashasebastian/Files/Classes/IV/Project/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2021_20241024.csv")


,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Emergency Department Indicator,Total Charges,Total Costs
0,New York City,Bronx,7000006.0,1169.0,Montefiore Medical Center - Henry & Lucy Moses...,70 or Older,104,M,Other Race,Spanish/Hispanic,...,Major,Extreme,Medical,Medicare,Medicaid,NaN,NaN,Y,"320,922.43","60,241.34"
1,New York City,Bronx,7000006.0,1169.0,Montefiore Medical Center - Henry & Lucy Moses...,50 to 69,104,F,White,Not Span/Hispanic,...,Moderate,Minor,Medical,Private Health Insurance,NaN,NaN,NaN,Y,"61,665.22","9,180.69"
2,New York City,Bronx,7000006.0,1168.0,Montefiore Medical Center-Wakefield Hospital,18 to 29,104,F,Other Race,Spanish/Hispanic,...,Minor,Minor,Surgical,Medicaid,NaN,NaN,NaN,N,"42,705.34","11,366.5"
3,New York City,Bronx,7000006.0,3058.0,Montefiore Med Center - Jack D Weiler Hosp of ...,70 or Older,104,M,Other Race,Spanish/Hispanic,...,Major,Major,Medical,Medicare,Medicaid,NaN,NaN,Y,"72,700.17","12,111.75"
4,New York City,Bronx,7000006.0,1169.0,Montefiore Medical Center - Henry & Lucy Moses...,50 to 69,104,F,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicare,Medicaid,NaN,NaN,Y,"55,562.51","8,339.72"


In [102]:
#Clean data 

# Columns to drop
columns_to_drop = ['Operating Certificate Number', 'Permanent Facility Id', 'Length of Stay', 'Discharge Year', 'CCSR Diagnosis Code', 'CCSR Procedure Code','APR DRG Code','APR MDC Code','APR Severity of Illness Code','APR Medical Surgical Description','Payment Typology 1','Payment Typology 2','Payment Typology 3','Birth Weight', 'Emergency Department Indicator']

# Dropping the irrelevant columns
df = df.drop(columns=columns_to_drop)
df.columns

Index(['Hospital Service Area', 'Hospital County', 'Facility Name',
       'Age Group', 'Zip Code - 3 digits', 'Gender', 'Race', 'Ethnicity',
       'Type of Admission', 'Patient Disposition',
       'CCSR Diagnosis Description', 'CCSR Procedure Description',
       'APR DRG Description', 'APR MDC Description',
       'APR Severity of Illness Description', 'APR Risk of Mortality',
       'Total Charges', 'Total Costs'],
      dtype='object')

In [86]:
#Diagnosis and Procedures

#Patient reason of admission description
# df['APR DRG Description'].unique()

#Diagnosis
# df['CCSR Diagnosis Description'].unique()

#Procedure 
# df['CCSR Procedure Description'].unique()

#Major Diagnostic Categories
# df['APR MDC Description'].unique()

#Severity of Illness
# df['APR Severity of Illness Description'].unique()

#Risk of Mortality
# df['APR Risk of Mortality'].unique()

array(['Extreme', 'Minor', 'Major', 'Moderate', nan], dtype=object)

In [163]:
# Removing rows with no data
df['APR Severity of Illness Description'].isna().sum()
df_cleaned = df.dropna(subset=['APR Severity of Illness Description'])
df_cleaned['CCSR Procedure Description'].isna().sum()
df_cleaned = df_cleaned.dropna(subset=['CCSR Procedure Description'])
df_cleaned = df_cleaned.dropna(subset=['Hospital County'])
df_cleaned.isna().sum()
df_cleaned.shape[0]

1547362

In [179]:
df_cleaned['Hospital County'].nunique()
# df['Hospital Service Area'].unique()
# df['Facility Name'].unique()
#Fetching location from csv according to county and state New York
df_cleaned['Merged'] = df_cleaned['Hospital County'] + ', ' + 'New York'

#list of all counties
counties = list(df_cleaned['Merged'].unique())
counties

['Bronx, New York',
 'Manhattan, New York',
 'Kings, New York',
 'Queens, New York',
 'Rockland, New York',
 'Westchester, New York',
 'Onondaga, New York',
 'Nassau, New York',
 'Otsego, New York',
 'Delaware, New York',
 'Sullivan, New York',
 'Orange, New York',
 'Ontario, New York',
 'Monroe, New York',
 'Columbia, New York',
 'Steuben, New York',
 'Cayuga, New York',
 'Albany, New York',
 'Madison, New York',
 'Warren, New York',
 'Yates, New York',
 'Cortland, New York',
 'Jefferson, New York',
 'Allegany, New York',
 'Schenectady, New York',
 'Lewis, New York',
 'St Lawrence, New York',
 'Erie, New York',
 'Oneida, New York',
 'Broome, New York',
 'Suffolk, New York',
 'Richmond, New York',
 'Oswego, New York',
 'Chautauqua, New York',
 'Putnam, New York',
 'Dutchess, New York',
 'Montgomery, New York',
 'Livingston, New York',
 'Fulton, New York',
 'Franklin, New York',
 'Ulster, New York',
 'Tompkins, New York',
 'Chemung, New York',
 'Cattaraugus, New York',
 'Niagara, New Yo

In [181]:
#Fetching latitude and longitude data of counties from OpenStreetMap
def fetch_coordinates(location):
    url = f"https://nominatim.openstreetmap.org/search?format=json&q={location}"
    headers = {"User-Agent": "HealthcareApp/2.0 (natasha.sebastian@gmail.com)"} 
    response = requests.get(url, headers=headers)
    data = response.json()
    if data:
        return data[0]['lat'], data[0]['lon']
    return None, None

# Output CSV file
output_file = "ny_counties_coordinates_1.csv"

# Fetch coordinates and save to CSV
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["County", "Latitude", "Longitude"])  # Header row

    for county in counties:
        lat, lon = fetch_coordinates(county)
        print(f"{county}: {lat}, {lon}")
        writer.writerow([county, lat, lon])

print(f"Coordinates saved to {output_file}")

Bronx, New York: 40.8466508, -73.8785937
Manhattan, New York: 40.7896239, -73.9598939
Kings, New York: 43.1509069, -73.8542895
Queens, New York: 40.6515585, -73.79267536317502
Rockland, New York: 41.1519319, -74.0357266
Westchester, New York: 41.1763139, -73.7907554
Onondaga, New York: 43.015598, -76.2257127
Nassau, New York: 40.7352994, -73.5615778
Otsego, New York: 42.5984272, -75.0142701
Delaware, New York: 42.194917, -75.0016302
Sullivan, New York: 41.7156311, -74.7804323
Orange, New York: 41.3873306, -74.2507287
Ontario, New York: 42.8580624, -77.295025
Monroe, New York: 41.3304767, -74.1866348
Columbia, New York: 42.2415027, -73.6723456
Steuben, New York: 42.2359045, -77.3750862
Cayuga, New York: 42.8093409, -76.5700777
Albany, New York: 42.6511674, -73.754968
Madison, New York: 42.875882, -75.6802581
Warren, New York: 43.5018687, -73.8164637
Yates, New York: 42.6444444, -77.112177
Cortland, New York: 42.5842136, -76.0704906
Jefferson, New York: 44.059311, -75.9995742
Allegany, N

In [203]:
df_cleaned['Gender'].unique()
df_filtered = df_cleaned[df_cleaned['Gender'] != 'U']
df_filtered['Ethnicity'].unique()
df_filtered = df_filtered[df_filtered['Ethnicity'] != 'Unknown']

In [213]:
df_filtered = df_filtered.drop(columns=['Merged'])

In [217]:
df_filtered.columns

Index(['Hospital Service Area', 'Hospital County', 'Facility Name',
       'Age Group', 'Zip Code - 3 digits', 'Gender', 'Race', 'Ethnicity',
       'Type of Admission', 'Patient Disposition',
       'CCSR Diagnosis Description', 'CCSR Procedure Description',
       'APR DRG Description', 'APR MDC Description',
       'APR Severity of Illness Description', 'APR Risk of Mortality',
       'Total Charges', 'Total Costs'],
      dtype='object')

In [229]:
# Rename columns in the DataFrame
df_filtered = df_filtered.rename(columns={
    'CCSR Diagnosis Description': 'Diagnosis Category',
    'CCSR Procedure Description': 'Procedure Category',
    'APR DRG Description': 'Diagnosis Group',
    'APR MDC Description': 'Major Diagnostic Category',
    'APR Severity of Illness Description': 'Illness Severity',
    'APR Risk of Mortality': 'Mortality Risk',
    'Total Charges': 'Billed Amount',
    'Total Costs': 'Total Hospital Costs'
})

In [231]:
# Display the updated DataFrame
df_filtered.head()

,Hospital Service Area,Hospital County,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Type of Admission,Patient Disposition,Diagnosis Category,Procedure Category,Diagnosis Group,Major Diagnostic Category,Illness Severity,Mortality Risk,Billed Amount,Total Hospital Costs
0,New York City,Bronx,Montefiore Medical Center - Henry & Lucy Moses...,70 or Older,104,M,Other Race,Spanish/Hispanic,Emergency,Home w/ Home Health Services,COVID-19,ISOLATION PROCEDURES,MAJOR RESPIRATORY INFECTIONS AND INFLAMMATIONS,DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM,Major,Extreme,"320,922.43","60,241.34"
2,New York City,Bronx,Montefiore Medical Center-Wakefield Hospital,18 to 29,104,F,Other Race,Spanish/Hispanic,Emergency,Home or Self Care,Previous C-section,CESAREAN SECTION,CESAREAN SECTION WITHOUT STERILIZATION,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",Minor,Minor,"42,705.34","11,366.5"
3,New York City,Bronx,Montefiore Med Center - Jack D Weiler Hosp of ...,70 or Older,104,M,Other Race,Spanish/Hispanic,Emergency,Home or Self Care,Urinary tract infections,ADMINISTRATION OF NUTRITIONAL AND ELECTROLYTIC...,KIDNEY AND URINARY TRACT INFECTIONS,DISEASES AND DISORDERS OF THE KIDNEY AND URINA...,Major,Major,"72,700.17","12,111.75"
4,New York City,Bronx,Montefiore Medical Center - Henry & Lucy Moses...,50 to 69,104,F,Black/African American,Not Span/Hispanic,Emergency,Home or Self Care,Paralysis (other than cerebral palsy),LUMBAR PUNCTURE,OTHER DISORDERS OF NERVOUS SYSTEM,DISEASES AND DISORDERS OF THE NERVOUS SYSTEM,Moderate,Minor,"55,562.51","8,339.72"
5,New York City,Bronx,Montefiore Med Center - Jack D Weiler Hosp of ...,50 to 69,105,M,Other Race,Spanish/Hispanic,Emergency,Home or Self Care,Complication of other surgical or medical care...,COMPUTERIZED TOMOGRAPHY (CT) WITHOUT CONTRAST,OTHER COMPLICATIONS OF TREATMENT,"INJURIES, POISONINGS AND TOXIC EFFECTS OF DRUGS",Major,Moderate,"109,269.27","18,443"


In [221]:
# Convert the NY Counties DataFrame to JSON format for easy integration with JavaScript
ny_counties_data = pd.read_csv('/Users/natashasebastian/Files/Classes/IV/Project/ny_counties_coordinates_1.csv')
ny_counties_data.head()
json_file_path = 'ny_counties_coordinates.json'
ny_counties_data.to_json(json_file_path, orient='records')
json_file_path

'ny_counties_coordinates.json'

In [233]:
# Save cleaned and filtered dataset
output_file_path = 'dataset.csv'  # Replace with your desired file name
df_filtered.to_csv(output_file_path, index=False)